In [1]:
import os
import pickle
from ibm_watson_studio_pipelines import WSPipelines

In [6]:
#@hidden_cell
## PROJECT COS 
ENDPOINT_URL_MLOPS = "https://s3.jp-tok.cloud-object-storage.appdomain.cloud"
AUTH_ENDPOINT = "https://iam.cloud.ibm.com/oidc/token"
API_KEY_MLOPS = "W7libKUAsvOKR9sQaYBzpQiReuiVp4Vm9JWqugRsW6Rq"
CRN_MLOPS = "crn:v1:bluemix:public:cloud-object-storage:global:a/694a501fd56b4f6b9aa747128ab5267b:5e28c0cc-7d72-49b7-be0b-4e0606310e4e::"
BUCKET_MLOPS  = "mlops-cpd-onprem"


cpd_api_key = os.getenv("cpd_api_key")

In [7]:
#@hidden_cell
# uncomment the apikey below for debugging
# apikey = "Ag2QXF-lYrasKCNs54ZFuv46emhdpa97JiusZz00Cl7U"

In [8]:
from ibm_botocore.client import Config
import ibm_boto3


def download_file_cos(local_file_name,key):
    cos = ibm_boto3.client(service_name='s3',
                           ibm_api_key_id=API_KEY_MLOPS,
                           ibm_service_instance_id=CRN_MLOPS,
                           ibm_auth_endpoint=AUTH_ENDPOINT,
                           config=Config(signature_version='oauth'),
                           endpoint_url=ENDPOINT_URL_MLOPS)
    try:
        res=cos.download_file(Bucket=BUCKET_MLOPS,Key=key,Filename=local_file_name)
    except Exception as e:
        print(Exception, e)
    else:
        print('File Downloaded')
        
def load_loss(key, filename):
    download_file_cos(key,filename)
    with open (filename,"rb") as f:
        loss = pickle.load(f)
    return loss

In [9]:
train_loss = load_loss('train_loss.pkl','train_loss.pkl')

File Downloaded


In [10]:
train_loss = train_loss['binary_logloss']

## Test if the model training converges

In [11]:
def check_loss_converge(loss):
    # check every 5 iterations if the loss has decreased
    coverges = [loss[i]>loss[i+4] for i in range(0, len(loss),5)]
    return all(coverges)

In [12]:
model_converges = check_loss_converge(train_loss)

In [13]:
model_converges

True

## Save the test results to pipeline

In [14]:
test_params = {'train_converge': model_converges}

In [16]:
import os
pipelines_client = WSPipelines.from_token(os.environ['USER_ACCESS_TOKEN'])
pipelines_client.store_results(test_params)

Running outside of Watson Studio Pipeline - storing results in the local filesystem for testing purposes...

  output paths:
    - "train_converge": .ibm_watson_studio_pipelines/results/train_converge


/opt/conda/envs/Python-3.10-Premium/lib/python3.10/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'internal-nginx-svc.cpd-instance.svc.cluster.local'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
